In [5]:
import requests
from bs4 import BeautifulSoup
import bs4
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import re

In [7]:
def getSearchPhrases():
    scope = ['https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
    client = gspread.authorize(creds)
    sheet = client.open("DIGHUM Sentiment")
    sheet = sheet.worksheet("searches")
    df = pd.DataFrame(sheet.get_all_values())
    if not df.empty:
        df.columns = df.iloc[0]
        df = df.iloc[1:]
    return df
searchPhrasesDf = getSearchPhrases()
searchPhrasesDf

,Phrases
1,Black Lives Matter
2,All Lives Matter
3,Blue Lives Matter


In [8]:
articlesDf = pd.DataFrame(columns=["query", "date","source", "title", "description"])
for query in searchPhrasesDf ["Phrases"]:
    print(query)
    preppedQuery = re.sub(" ", "%20", query)
    url = "https://news.google.com/rss/search?q=" + preppedQuery + "when:1d"
    print(url)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html5lib') 
    dates = soup.find_all('pubdate')
    titles = soup.find_all('title')
    descriptions = soup.find_all('description')
    sources = soup.find_all('source')
    for x in range(1, len(dates) + 1):
        articlesDf = articlesDf.append({"query" : query,
                                        "date" : dates[x - 1].getText(),
                                        "source": sources[x - 1].next_sibling,
                                        'title': titles[x].getText(), 
                                        "description": re.split('>', descriptions[x].get_text())[1][:-3]}, ignore_index=True)
articlesDf

Black Lives Matter
https://news.google.com/rss/search?q=Black%20Lives%20Matterwhen:1d
All Lives Matter
https://news.google.com/rss/search?q=All%20Lives%20Matterwhen:1d
Blue Lives Matter
https://news.google.com/rss/search?q=Blue%20Lives%20Matterwhen:1d


,query,date,source,title,description
0,Black Lives Matter,"Wed, 01 Jul 2020 19:38:38 GMT",NPR,Trump Calls NYC Black Lives Matter Plan A 'Sym...,
1,Black Lives Matter,"Wed, 01 Jul 2020 16:42:00 GMT",NBC News,Vice urges advertisers to stop blocking 'Black...,
2,Black Lives Matter,"Wed, 01 Jul 2020 14:03:46 GMT",MarketWatch,People donated millions of dollars to the wron...,People donated millions of dollars to the wron...
3,Black Lives Matter,"Wed, 01 Jul 2020 17:42:53 GMT",NBC News,Target customer confronts employee in Black Li...,Target customer confronts employee in Black Li...
4,Black Lives Matter,"Wed, 01 Jul 2020 12:35:29 GMT",The Nation,Black Lives Matter: Walking Forward - The Nation,Black Lives Matter: Walking Forward
...,...,...,...,...,...
256,Blue Lives Matter,"Tue, 30 Jun 2020 22:24:00 GMT",WOODTV.com,Rockford man charged with ethnic intimidation ...,Rockford man charged with ethnic intimidation
257,Blue Lives Matter,"Tue, 30 Jun 2020 22:10:18 GMT",Courier Post,"Fourth of July: Where to find fireworks, parad...","Fourth of July: Where to find fireworks, parad..."
258,Blue Lives Matter,"Wed, 01 Jul 2020 00:12:56 GMT",Blue Mountains Gazette,SA service to reduce deaths in custody - Blue ...,SA service to reduce deaths in custody
259,Blue Lives Matter,"Wed, 01 Jul 2020 12:42:08 GMT",KSTP,Trump tweets that Russia bounty allegations ar...,Trump tweets that Russia bounty allegations ar...


In [9]:
articlesDf

,query,date,source,title,description
0,Black Lives Matter,"Wed, 01 Jul 2020 19:38:38 GMT",NPR,Trump Calls NYC Black Lives Matter Plan A 'Sym...,
1,Black Lives Matter,"Wed, 01 Jul 2020 16:42:00 GMT",NBC News,Vice urges advertisers to stop blocking 'Black...,
2,Black Lives Matter,"Wed, 01 Jul 2020 14:03:46 GMT",MarketWatch,People donated millions of dollars to the wron...,People donated millions of dollars to the wron...
3,Black Lives Matter,"Wed, 01 Jul 2020 17:42:53 GMT",NBC News,Target customer confronts employee in Black Li...,Target customer confronts employee in Black Li...
4,Black Lives Matter,"Wed, 01 Jul 2020 12:35:29 GMT",The Nation,Black Lives Matter: Walking Forward - The Nation,Black Lives Matter: Walking Forward
...,...,...,...,...,...
256,Blue Lives Matter,"Tue, 30 Jun 2020 22:24:00 GMT",WOODTV.com,Rockford man charged with ethnic intimidation ...,Rockford man charged with ethnic intimidation
257,Blue Lives Matter,"Tue, 30 Jun 2020 22:10:18 GMT",Courier Post,"Fourth of July: Where to find fireworks, parad...","Fourth of July: Where to find fireworks, parad..."
258,Blue Lives Matter,"Wed, 01 Jul 2020 00:12:56 GMT",Blue Mountains Gazette,SA service to reduce deaths in custody - Blue ...,SA service to reduce deaths in custody
259,Blue Lives Matter,"Wed, 01 Jul 2020 12:42:08 GMT",KSTP,Trump tweets that Russia bounty allegations ar...,Trump tweets that Russia bounty allegations ar...


In [5]:
scope = ['https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
client = gspread.authorize(creds)
sheet = client.open("DIGHUM Sentiment")
sheet = sheet.worksheet("GoogleNews")
df = pd.DataFrame(sheet.get_all_values())
if not df.empty:
    df.columns =  df.iloc[0]
    df = df.iloc[1:]
newDfForSheet = articlesDf.append(df)
newindex = [x for x in range(len(newDfForSheet["query"]))]
newDfForSheet["check"] = newindex
newDfForSheet = newDfForSheet.set_index("check")
spreadsheet_key = '1YgFoSOEt9edjE0CTmMayAN6J09zMHdI-WQu9BL758Ek'
wks_name = "GoogleNews"
d2g.upload(newDfForSheet, spreadsheet_key, wks_name, credentials=creds, row_names = False)

<Worksheet 'GoogleNews' id:1652961889>